In [1]:
import sys, datetime
sys.path.append("../../scripts/")
from s3_support import *

import pandas as pd
import numpy as np

# 1. org donor retention

## 0. Unique donors & volume

In [2]:
q = '''select
            year,
            count(distinct(email)) as donors
        from transactions
        where status='A' 
        group by year'''
donors = redshift_query_read(q, schema='production')

In [1]:
print("{:,} entries".format(len(donors)))
print("{} to {} years".format(donors['year'].min(), donors['year'].max()))

NameError: name 'donors' is not defined

In [3]:
print("Systen wide unique donor growth by calendar year")

donors.sort_values('year', ascending=True, inplace=True)
donors['pct_change'] = donors['donors'].pct_change()
donors.tail()

,year,donors,pct_change
18,2020,1034617,0.297171
13,2021,1061986,0.026453
9,2022,1176579,0.107904
11,2023,1325353,0.126446
17,2024,1385971,0.045737


In [4]:
q = '''select
            year,
            count(distinct(org)) as orgs,
            sum(amount) as vol,
            sum(donations_amt) as vol_donations,
            sum(events_amt) as vol_events,
            sum(registrations_amt) as vol_registrations
        from transactions
        where status='A' 
        group by year'''
vol = redshift_query_read(q, schema='production')

In [ ]:
print("{:,} entries".format(len(vol)))
print("{:,} peak orgs".format(vol['org'].max()))
print("{} to {} years".format(vol['year'].min(), vol['year'].max()))

In [5]:
vol.sort_values('year', ascending=True, inplace=True)
vol['vol/org'] = vol['vol'] / vol['orgs']
vol['vol/org_change'] = vol['vol/org'].pct_change()
vol['orgs_change'] = vol['orgs'].pct_change()
vol['vol_change'] = vol['vol'].pct_change()
vol['donations_change'] = vol['vol_donations'].pct_change()
vol['events_change'] = vol['vol_events'].pct_change()
vol['registration_change'] = vol['vol_registrations'].pct_change()
vol[['year'] + [c for c in vol.columns if 'change' in c]].tail()

,year,vol/org_change,orgs_change,vol_change,donations_change,events_change,registration_change
17,2020,0.231009,0.122363,0.381639,0.497577,-0.381877,-0.419688
18,2021,0.058671,0.116831,0.182356,0.126660,0.694446,0.320317
3,2022,0.055123,0.074055,0.133260,0.071024,0.565177,0.487702
10,2023,0.069867,0.084378,0.160140,0.115844,0.330906,0.344901
15,2024,-0.337682,0.462872,-0.031113,-0.098289,0.409246,0.043609


### 0.a qgiv x bloomerang

#### donors

In [6]:
q = '''select
            year,
            count(distinct(email)) as donors
        from transactions
        where 
            status='A' and
            org in (select distinct(org) from syslog_logs where message ilike '%bloomerang%')
        group by year'''
qb_donors = redshift_query_read(q, schema='production')

In [7]:
qb_donors.sort_values('year', ascending=True, inplace=True)
qb_donors['pct_change'] = qb_donors['donors'].pct_change()
qb_donors.tail()

,year,donors,pct_change
16,2020,179909,0.666781
17,2021,206594,0.148325
15,2022,253217,0.225675
7,2023,285886,0.129016
5,2024,453767,0.587231


__2022 only orgs YoY__

In [8]:
q = '''select
            year,
            count(distinct(org)) as orgs,
            count(distinct(email)) as donors
        from transactions
        where 
            status='A' and
            org in (select distinct(org) from syslog_logs 
                    where 
                        message ilike '%bloomerang%' and
                        date_part('year', created)=2022)
        group by year'''
qb_donors = redshift_query_read(q, schema='production')

In [9]:
qb_donors['donors/orgs'] = qb_donors['donors'] / qb_donors['orgs']

In [10]:
qb_donors.sort_values('year', ascending=True, inplace=True)
qb_donors['pct_change'] = qb_donors['donors'].pct_change()
qb_donors['d/o_change'] = qb_donors['donors/orgs'].pct_change()
qb_donors.tail()

,year,orgs,donors,donors/orgs,pct_change,d/o_change
17,2020,212,106001,500.004717,0.443703,0.028298
18,2021,303,138635,457.541254,0.307865,-0.084926
15,2022,440,168566,383.104545,0.215898,-0.162689
6,2023,377,166113,440.618037,-0.014552,0.150125
3,2024,337,147849,438.721068,-0.109949,-0.004305


#### volume

In [11]:
q = '''select
            year,
            count(distinct(org)) as orgs,
            sum(amount) as vol,
            sum(donations_amt) as vol_donations,
            sum(events_amt) as vol_events,
            sum(registrations_amt) as vol_registrations
        from transactions
        where 
            status='A' and
            org in (select distinct(org) from syslog_logs where message ilike '%bloomerang%')
        group by year'''
qb_vol = redshift_query_read(q, schema='production')

In [12]:
qb_vol.sort_values('year', ascending=True, inplace=True)
qb_vol['vol/org'] = qb_vol['vol'] / qb_vol['orgs']
qb_vol['vol/org_change'] = qb_vol['vol/org'].pct_change()
qb_vol['orgs_change'] = qb_vol['orgs'].pct_change()
qb_vol['vol_change'] = qb_vol['vol'].pct_change()
qb_vol['donations_change'] = qb_vol['vol_donations'].pct_change()
qb_vol['events_change'] = qb_vol['vol_events'].pct_change()
qb_vol['registration_change'] = qb_vol['vol_registrations'].pct_change()
qb_vol[['year'] + [c for c in qb_vol.columns if 'change' in c]].tail()

,year,vol/org_change,orgs_change,vol_change,donations_change,events_change,registration_change
18,2020,0.184100,0.419708,0.681076,0.818074,-0.189114,-0.250391
15,2021,0.210825,0.293059,0.565668,0.392643,1.177128,0.576077
14,2022,0.009577,0.248509,0.260466,0.153867,0.536365,0.833417
7,2023,0.066105,0.135350,0.210402,0.138205,0.499185,0.379754
10,2024,-0.594759,2.478261,0.409535,0.260777,1.315853,0.180773


__2022 only orgs__

In [13]:
q = '''select
            year,
            count(distinct(org)) as orgs,
            sum(amount) as vol,
            sum(donations_amt) as vol_donations,
            sum(events_amt) as vol_events,
            sum(registrations_amt) as vol_registrations
        from transactions
        where 
            status='A' and
            org in (select distinct(org) from syslog_logs 
                    where 
                        message ilike '%bloomerang%' and
                        date_part('year', created)=2022)
        group by year'''
qb_vol = redshift_query_read(q, schema='production')

In [14]:
qb_vol.sort_values('year', ascending=True, inplace=True)
qb_vol['vol/org'] = qb_vol['vol'] / qb_vol['orgs']
qb_vol['vol/org_change'] = qb_vol['vol/org'].pct_change()
qb_vol['orgs_change'] = qb_vol['orgs'].pct_change()
qb_vol['vol_change'] = qb_vol['vol'].pct_change()
qb_vol['donations_change'] = qb_vol['vol_donations'].pct_change()
qb_vol['events_change'] = qb_vol['vol_events'].pct_change()
qb_vol['registration_change'] = qb_vol['vol_registrations'].pct_change()
qb_vol[['year'] + [c for c in qb_vol.columns if 'change' in c]].tail()

,year,vol/org_change,orgs_change,vol_change,donations_change,events_change,registration_change
13,2020,0.145885,0.403974,0.608793,0.727589,-0.124742,-0.288921
14,2021,-0.099280,0.429245,0.287349,0.184347,0.898683,0.666912
7,2022,-0.081450,0.452145,0.333868,0.186652,0.654302,1.185331
15,2023,0.266062,-0.143182,0.084785,0.047906,0.251829,0.208486
11,2024,-0.136573,-0.106101,-0.228183,-0.270304,-0.024031,-0.257191


## 1. data build, table management

In [15]:
q = '''drop table if exists public.donor_retention_orgs'''
redshift_query_write(q, schema='public')

In [16]:
# org retention
q = '''select
           org,
           email as donor,
           year,
           count(id) as transactions,
           sum(amount) as vol,
           count(distinct(case when recurring=0 then id else null end)) as onetime,
           count(distinct(recurring)) as recurring,
           sum(purchases_quantity) as purchases,
           sum(donations_count) as donations,
           sum(events_count) as events,
           sum(registrations_count) as registrations,
           sum(auctiondonation_count) as auctiondonations,
           sum(auctionpurchase_count) as auctionpurchases,
           sum(gift_assist_count) as giftassist,
           sum(matchinggifts_count) as matchinggifts
        into public.donor_retention_orgs
        from production.transactions
        where status='A'
        group by org, email, year;'''
redshift_query_write(q, schema='production')

In [17]:
q = "select * from donor_retention_orgs"
org_donors = redshift_query_read(q, schema='public')

In [18]:
org_donors['recurring'] = org_donors['recurring']-1
org_donors.tail(3)

,org,donor,year,transactions,vol,onetime,recurring,purchases,donations,events,registrations,auctiondonations,auctionpurchases,giftassist,matchinggifts
9337836,808,26457ed929081ac89bd00f433592124783818756,2023,1,0.00,1,0,0,0,1,0,0,0,0,0
9337837,444481,645b9cb1aa29b9b064b0494e70a3ccf1e3564901,2020,1,271.05,1,0,0,1,0,0,0,0,1,0
9337838,445763,5bdbbabd95a680bc876a24245306ded925413c47,2020,1,0.00,1,0,0,0,0,1,0,0,0,0
9337839,441359,560e6f6a79353ff217a104615bb463accae2a799,2024,1,53.00,1,0,0,1,0,0,0,0,1,0
9337840,451161,09a4413546c058e673ee03f1f31850a75963965b,2024,1,208.00,1,0,0,1,0,0,0,0,1,0


In [19]:
print("{:,} rows".format(len(org_donors)))
print("{:,} unique orgs".format(len(org_donors['org'].unique())))
print("{:,} unique donors".format(len(org_donors['donor'].unique())))
print("{} to {}".format(org_donors['year'].min(), org_donors['year'].max()))

11,216 unique orgs; 6,369,357 unique donors
2006 to 2024


## 2. exploration: average donor

In [20]:
agg_dict = {'year': 'count', 'transactions': 'sum', 'recurring': 'sum'}
org_retention_donors = org_donors.groupby(['donor', 'org']).agg(agg_dict).reset_index()

org_retention = org_retention_donors.groupby('org')[['year', 'transactions', 'recurring']].agg(['mean', 'median', 'count']).reset_index()
org_retention.mean()

org                     372989.532097
year          mean           1.164342
              median         1.013864
              count        607.981366
transactions  mean           3.358536
              median         2.225704
              count        607.981366
recurring     mean           0.040933
              median         0.004235
              count        607.981366
dtype: float64

In [21]:
print("All orgs ({:,}):".format(len(org_retention)))
print(org_retention.drop('org', axis=1).mean())

print()

print("Orgs w/ > 500 donors ({:,}):".format(len(org_retention[org_retention['year']['count']>500])))
print(org_retention[org_retention['year']['count']>500].drop('org', axis=1).mean())

print()

print("Orgs w/ > 1k donors ({:,}):".format(len(org_retention[org_retention['year']['count']>1000])))
print(org_retention[org_retention['year']['count']>1000].drop('org', axis=1).mean())

print()

print("Orgs w/ > 5k donors ({:,}):".format(len(org_retention[org_retention['year']['count']>5000])))
print(org_retention[org_retention['year']['count']>5000].drop('org', axis=1).mean())

All orgs (11,216):
year          mean        1.164342
              median      1.013864
              count     607.981366
transactions  mean        3.358536
              median      2.225704
              count     607.981366
recurring     mean        0.040933
              median      0.004235
              count     607.981366
dtype: float64

Orgs w/ > 500 donors (2,131):
year          mean         1.342465
              median       1.010793
              count     2797.885500
transactions  mean         2.721831
              median       1.072267
              count     2797.885500
recurring     mean         0.049485
              median       0.001408
              count     2797.885500
dtype: float64

Orgs w/ > 1k donors (1,252):
year          mean         1.366552
              median       1.008786
              count     4264.476837
transactions  mean         2.732643
              median       1.066294
              count     4264.476837
recurring     mean         0.045631

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [22]:
# recurring donor vs onetime retention
rec_donors = org_donors[org_donors['recurring']!=0]
onetime_donors = org_donors[org_donors['recurring']==0]

In [23]:
rec_donors.groupby(['org', 'donor']).agg({'year': 'count', 'transactions': 'sum'}).reset_index()[['year', 'transactions']].agg(['mean', 'median'])

,year,transactions
mean,1.455058,20.35803
median,1.000000,9.00000


In [24]:
onetime_donors.groupby(['org', 'donor']).agg({'year': 'count', 'transactions': 'sum'}).reset_index()[['year', 'transactions']].agg(['mean', 'median'])

,year,transactions
mean,1.340849,2.527036
median,1.000000,1.000000


## 3. retention by org performance

In [25]:
orgs = org_retention[['org', 'year']]
orgs.columns = ['org', 'mean', 'median', 'count']
orgs.tail(3)

,org,mean,median,count
11213,455585,1.0,1.0,1
11214,455595,1.0,1.0,1
11215,455599,1.0,1.0,1


In [26]:
q = "select org, sum(amount) from transactions where status='A' group by org"
org_perf = redshift_query_read(q, schema='production')

In [27]:
org_perf = org_perf.merge(orgs[['org', 'mean']])
org_perf.columns = ['org', 'volume', 'retention']
org_perf.tail(3)

,org,volume,retention
11213,454225,1.0,1.0
11214,453337,1.0,1.0
11215,455428,1.0,1.0


In [28]:
print("$0 to $500:")
print("\t{:,} orgs".format(len(org_perf[org_perf['volume']<=500])))
print("\t{:.4f} mean retention".format(org_perf[org_perf['volume']<=500]['retention'].mean()))
print("\t{:.4f} median retention".format(org_perf[org_perf['volume']<=500]['retention'].median()))

print("$500 to $5,000:")
print("\t{:,} orgs".format(len(org_perf[(org_perf['volume']>500)&(org_perf['volume']<=5000)])))
print("\t{:.4f} mean retention".format(org_perf[(org_perf['volume']>500)&(org_perf['volume']<=5000)]['retention'].mean()))
print("\t{:.4f} median retention".format(org_perf[(org_perf['volume']>500)&(org_perf['volume']<=5000)]['retention'].median()))

print("$5,000 to $50,000:")
print("\t{:,} orgs".format(len(org_perf[(org_perf['volume']>5000)&(org_perf['volume']<=50000)])))
print("\t{:.4f} mean retention".format(org_perf[(org_perf['volume']>5000)&(org_perf['volume']<=50000)]['retention'].mean()))
print("\t{:.4f} median retention".format(org_perf[(org_perf['volume']>5000)&(org_perf['volume']<=50000)]['retention'].median()))

print("$50,000 to $250,000:")
print("\t{:,} orgs".format(len(org_perf[(org_perf['volume']>50000)&(org_perf['volume']<=250000)])))
print("\t{:.4f} mean retention".format(org_perf[(org_perf['volume']>50000)&(org_perf['volume']<=250000)]['retention'].mean()))
print("\t{:.4f} median retention".format(org_perf[(org_perf['volume']>50000)&(org_perf['volume']<=250000)]['retention'].median()))

print("$250,000 to $500,000:")
print("\t{:,} orgs".format(len(org_perf[(org_perf['volume']>250000)&(org_perf['volume']<=500000)])))
print("\t{:.4f} mean retention".format(org_perf[(org_perf['volume']>250000)&(org_perf['volume']<=500000)]['retention'].mean()))
print("\t{:.4f} median retention".format(org_perf[(org_perf['volume']>250000)&(org_perf['volume']<=500000)]['retention'].median()))

print("$500,000+:")
print("\t{:,} orgs".format(len(org_perf[(org_perf['volume']>500000)])))
print("\t{:.4f} mean retention".format(org_perf[(org_perf['volume']>500000)]['retention'].mean()))
print("\t{:.4f} median retention".format(org_perf[(org_perf['volume']>500000)]['retention'].median()))

$0 to $500:
	2,384 orgs
	1.0161 mean retention
	1.0000 median retention
$500 to $5,000:
	1,639 orgs
	1.0535 mean retention
	1.0000 median retention
$5,000 to $50,000:
	3,305 orgs
	1.1166 mean retention
	1.0256 median retention
$50,000 to $250,000:
	2,245 orgs
	1.2477 mean retention
	1.1878 median retention
$250,000 to $500,000:
	670 orgs
	1.3948 mean retention
	1.3297 median retention
$500,000+:
	973 orgs
	1.5257 mean retention
	1.4278 median retention


It's clear that higher performing org's have a better retention. We cannot determine from the above information alone that the relationship is causal, and very well may be a inverse (ie, org's with greater retention raise more rather than org's that raise more have greater retention). 

_We will need to examine retention over time in order to see if the higher performing org's had their greater retention from periods of time in which they did not raise more funds. This could be a good indicator of future growth._

### org retention YoY

In [48]:
org_year_data = []
for o in org_donors['org'].unique().tolist():
    _org_df = org_donors[org_donors['org']==o]
    _org_years = sorted(_org_df['year'].unique().tolist())
    
    if len(_org_years) > 1:
        for year in _org_years:
            if year - 1 in _org_years:
                _this_year = _org_df[_org_df['year']==year]
                _last_year = _org_df[_org_df['year']==(year - 1)]
                _all_prior_years = _org_df[_org_df['year']<year]
                _all_prior_years_plus = _org_df[_org_df['year']<year-1]
                
                new_donors_last_year = _last_year[~_last_year['donor'].isin(_all_prior_years_plus['donor'])]
                
                org_year_data.append({
                    'org': o,
                    'year': year,
                    'donors_unique': len(_this_year['donor']),
                    'donors_retained': len(_this_year[_this_year['donor'].isin(_last_year['donor'])]),
                    'new_donors': len(_this_year[~_this_year.isin(_all_prior_years['donor'])]),
                    'new_donors_retained': len(new_donors_last_year[new_donors_last_year['donor'].isin(_this_year['donor'])]),
                    'new_donors_last_year': len(new_donors_last_year),
                    'recurring_donors': _this_year['recurring'].sum()
                })

In [50]:
org_year_df = pd.DataFrame(org_year_data)
org_year_df['retention'] = org_year_df['donors_retained'] / org_year_df['donors_unique']
org_year_df['new_donor_retention'] = org_year_df['new_donors_retained'] / org_year_df['new_donors_last_year']

In [31]:
print("Average annual retention, new donors, recurring per org")
org_year_df.groupby('year')[['retention', 'new_donors', 'recurring_donors']].agg(['mean', 'median']).reset_index()

Average annual retention, new donors, recurring per org


year retention            new_donors        recurring_donors       
              mean    median        mean median             mean median
0   2007  0.116667  0.116667   10.500000   10.5         1.500000    1.5
1   2008  0.188858  0.110957  102.361111   26.5         5.972222    1.0
2   2009  0.178926  0.112374  128.400000   56.5         6.113333    1.0
3   2010  0.192114  0.149485  131.684211   52.0         5.987616    0.0
4   2011  0.203230  0.154867  119.051392   44.0         6.402570    1.0
5   2012  0.245760  0.186047  138.184087   54.0         8.533541    1.0
6   2013  0.259501  0.210526  176.061358   60.0        12.279373    1.0
7   2014  0.264081  0.224000  186.108716   52.0        15.253046    2.0
8   2015  0.297470  0.244048  234.039578   64.0        17.188215    2.0
9   2016  0.310906  0.254084  199.113150   64.0        13.154434    1.0
10  2017  0.303459  0.250000  254.909339   67.0        13.062713    1.0
11  2018  0.297518  0.242424  289.554505   67.0        12.671301    1.0
12  2019  0.295196  0.242788  320.094751   66.0        11.721975    1.0
13  2020  0.283932  0.220000  371.864865   69.0        13.537799    1.0
14  2021  0.315123  0.250281  340.367637   75.0        13.995842    0.0
15  2022  0.330881  0.268501  343.142632   79.5        13.373741    0.0
16  2023  0.337041  0.272727  356.224242   82.0        16.402597    0.0
17  2024  0.357468  0.297250  315.431903   68.0        13.246113    0.0

### qgiv x bloomerang

In [32]:
q = "select distinct(org) as org from syslog_logs where message ilike '%bloomerang%'"
qb_orgs = redshift_query_read(q, schema='production')['org'].tolist()

In [33]:
org_year_df[org_year_df['org'].isin(qb_orgs)].groupby('year')['retention'].agg(['mean', 'median']).reset_index().tail()

,year,mean,median
13,2020,0.230159,0.203911
14,2021,0.276497,0.237113
15,2022,0.292441,0.250000
16,2023,0.291641,0.248053
17,2024,0.307822,0.269653


__2022 orgs only__

In [34]:
q = '''select distinct(org) as org from syslog_logs 
        where 
            message ilike '%bloomerang%' and
            date_part('year', created)=2022'''
qb_orgs_2022 = redshift_query_read(q, schema='production')['org'].tolist()

In [35]:
org_year_df[org_year_df['org'].isin(qb_orgs_2022)].groupby('year')['retention'].agg(['mean', 'median', 'nunique']).reset_index().tail()

,year,mean,median,nunique
13,2020,0.205501,0.197842,138
14,2021,0.253102,0.232810,197
15,2022,0.272231,0.245473,259
16,2023,0.282295,0.250749,329
17,2024,0.345744,0.303867,281


### orgs segments

In [52]:
q = """select
            org,
            sum(amount) as volume
        from transactions
        where status='A' 
        group by org"""
#org_perf = redshift_query_read(q, schema='production')

In [36]:
orgs_500k = org_perf[(org_perf['volume']>500000)]['org'].tolist()
orgs_250k = org_perf[(org_perf['volume']>250000)&(org_perf['volume']<=500000)]['org'].tolist()
orgs_50k = org_perf[(org_perf['volume']>50000)&(org_perf['volume']<=250000)]['org'].tolist()

In [37]:
def get_rank(org):
    if org in orgs_500k:
        return '500k'
    elif org in orgs_250k:
        return '250k'
    elif org in orgs_50k:
        return '50k'
    return None

org_year_df['rank'] = org_year_df['org'].apply(get_rank)

In [38]:
org_year_df.groupby(['year', 'rank'])['retention'].agg(['mean', 'median']).reset_index().pivot(index='year', columns='rank', values=['mean', 'median'])

mean                        median                    
rank      250k      500k       50k      250k      500k       50k
year                                                            
2007       NaN  0.116667       NaN       NaN  0.116667       NaN
2008  0.291248  0.085013  0.276362  0.138591  0.064516  0.200000
2009  0.207967  0.140444  0.153517  0.172078  0.128778  0.099578
2010  0.198626  0.179593  0.168608  0.173913  0.149485  0.165217
2011  0.247110  0.202267  0.205484  0.180000  0.152985  0.172671
2012  0.256264  0.232395  0.231625  0.208589  0.194737  0.184211
2013  0.271062  0.244622  0.269636  0.245449  0.220315  0.228261
2014  0.267967  0.255427  0.275361  0.250000  0.232057  0.237891
2015  0.328992  0.288054  0.296829  0.257071  0.265146  0.230271
2016  0.328646  0.307167  0.306668  0.259084  0.280000  0.250000
2017  0.313360  0.311031  0.290147  0.251298  0.286903  0.235294
2018  0.314036  0.312336  0.296676  0.259843  0.273346  0.244269
2019  0.317714  0.317563  0.296635  0.257353  0.281106  0.238860
2020  0.291906  0.284721  0.298393  0.230984  0.244016  0.220000
2021  0.325775  0.349049  0.308709  0.267343  0.312917  0.238966
2022  0.339368  0.349695  0.317235  0.279188  0.317891  0.254545
2023  0.358371  0.370299  0.334602  0.285469  0.327273  0.269703
2024  0.373225  0.402895  0.351619  0.319502  0.349326  0.284993

In [39]:
org_year_df.groupby(['year', 'rank'])['donors_unique'].agg(['mean', 'median']).reset_index().pivot(index='year', columns='rank', values=['mean', 'median'])

mean                          median             
rank        250k         500k         50k   250k   500k   50k
year                                                         
2007         NaN    10.500000         NaN    NaN   10.5   NaN
2008  228.500000   116.769231   81.428571  148.5   62.0  11.0
2009  171.904762   217.955556  119.333333   90.0  100.0  63.5
2010  153.228571   231.701031  131.584270   88.0  110.0  54.0
2011  160.127660   230.585185   97.643939   66.0  136.0  50.5
2012  152.890411   299.742857  102.513661   84.0  156.0  65.0
2013  226.072917   373.126126  105.511111   74.0  197.0  59.0
2014  173.297872   455.376997   80.291667   79.0  202.0  46.0
2015  140.343373   568.584239   82.682635   78.0  228.5  46.5
2016  180.945000   444.534606   83.797872   80.0  219.0  45.5
2017  171.646512   591.067347   91.334873   90.0  264.0  46.0
2018  199.752896   720.876786  101.431559  102.0  285.5  48.0
2019  217.880259   845.745789  121.217523  109.0  332.0  52.0
2020  231.225543  1066.558496  108.503294  133.0  407.0  52.0
2021  240.975610   988.822917  116.261959  146.0  375.0  61.0
2022  252.146067  1021.844140  131.114738  159.0  408.5  74.0
2023  294.781893  1092.913899  148.551601  182.0  434.0  77.0
2024  294.370221   962.473039  159.958949  150.0  358.0  74.0

Retention does appear to be consistent year over year as would be expected among top performers, with retention degrading as performance decreases for the organization. The above is annual averages for all time performance for the given organization. We need to look at averages year over year by age (org year 1, year 2, etc.) rather than calendar years to confirm, but one would assume that if retention is a key indicator for a top performing organization it would be present over time. 

In [51]:
org_year_df.to_csv("org_year.csv", index=False)

# 2. form retention

## 1. data build, table management

In [ ]:
q = '''drop table if exists donor_retention_forms'''
redshift_query_write(q, schema='public')

In [ ]:
# form retention
q = '''select
           t.org,
           t.form,
           f.type as formtype,
           t.email as donor,
           t.year,
           count(t.id) as transactions,
           sum(t.amount) as vol,
           count(distinct(t.recurring)) as recurring_origin,
           count(distinct(case when recuring!=0 then id else null end)) as recurring,
           sum(t.purchases_quantity) as purchases,
           sum(t.donations_count) as donations,
           sum(t.events_count) as events,
           sum(t.registrations_count) as registrations,
           sum(t.auctiondonation_count) as auctiondonations,
           sum(t.auctionpurchase_count) as auctionpurchases,
           sum(t.gift_assist_count) as giftassist,
           sum(t.matchinggifts_count) as matchinggifts
        into public.donor_retention_forms
        from production.transactions as t
            left join production.form as f on t.form=f.id
        where t.status='A'
        group by t.org, t.form, t.email, t.year, f.type;'''
redshift_query_write(q, schema='production')

In [4]:
q = "select * from donor_retention_forms"
form_donors = redshift_query_read(q, schema='public')

In [56]:
form_donors['recurring'] = form_donors['recurring']-1
form_donors.tail()

,org,form,donor,year,transactions,vol,recurring,purchases,donations,events,registrations,auctiondonations,auctionpurchases,giftassist,matchinggifts
5999122,442789,975880,e48e33fb9449bc3f7410cee28acce6408e486e09,2021,2,52.5,0,0,1,0,1,0,0,0,0
5999123,444181,961787,c5a464e801b64f0fae061f03d76452d64779263f,2022,1,25.0,0,0,1,0,0,0,0,0,0
5999124,444188,946875,575861db1f1ba41a9db5674dcc906bc18d73f2d8,2019,3,201.4,0,0,3,0,0,0,0,0,0
5999125,31752,99394,af1b19b653b67824a8f98a22b0d8e2c5547e20d7,2020,1,15.0,0,0,0,1,0,0,0,0,0
5999126,1965,91193,adb94788d00c8ae8c2f671fcbea30ff82fcf1916,2022,1,0.0,0,0,0,1,0,0,0,0,0


## 2. exploration

In [60]:
agg_dict = {'year': 'count', 'transactions': 'sum', 'recurring': 'sum'}
form_retention_donors = form_donors.groupby(['donor', 'form']).agg(agg_dict).reset_index()

form_retention = form_retention_donors.groupby('form')[['year', 'transactions', 'recurring']].agg(['mean', 'median', 'count']).reset_index()
form_retention.mean()

form                    870266.055196
year          mean           1.092001
              median         1.034565
              count        136.146333
transactions  mean           2.224225
              median         1.634519
              count        136.146333
recurring     mean           0.035886
              median         0.014555
              count        136.146333
dtype: float64

In [61]:
print("All forms ({:,}):".format(len(form_retention)))
print(form_retention.drop('form', axis=1).mean())

print()

print("Forms w/ > 500 donors ({:,}):".format(len(form_retention[form_retention['year']['count']>500])))
print(form_retention[form_retention['year']['count']>500].drop('form', axis=1).mean())

print()

print("Forms w/ > 1k donors ({:,}):".format(len(form_retention[form_retention['year']['count']>1000])))
print(form_retention[form_retention['year']['count']>1000].drop('form', axis=1).mean())

print()

print("Forms w/ > 5k donors ({:,}):".format(len(form_retention[form_retention['year']['count']>5000])))
print(form_retention[form_retention['year']['count']>5000].drop('form', axis=1).mean())

All forms (37,032):
year          mean        1.092001
              median      1.034565
              count     136.146333
transactions  mean        2.224225
              median      1.634519
              count     136.146333
recurring     mean        0.035886
              median      0.014555
              count     136.146333
dtype: float64

Forms w/ > 500 donors (1,863):
year          mean         1.210714
              median       1.013956
              count     1606.089640
transactions  mean         2.400871
              median       1.174718
              count     1606.089640
recurring     mean         0.042604
              median       0.004294
              count     1606.089640
dtype: float64

Forms w/ > 1k donors (834):
year          mean         1.250527
              median       1.015588
              count     2720.266187
transactions  mean         2.689519
              median       1.245803
              count     2720.266187
recurring     mean         0.04882

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4147: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [57]:
# recurring donor vs onetime retention
rec_donors = form_donors[form_donors['recurring']!=0]
onetime_donors = form_donors[form_donors['recurring']==0]

In [58]:
rec_donors.groupby(['org', 'donor']).agg({'year': 'count', 'transactions': 'sum'}).reset_index()[['year', 'transactions']].agg(['mean', 'median'])

,year,transactions
mean,1.36738,17.022959
median,1.00000,7.000000


In [59]:
onetime_donors.groupby(['org', 'donor']).agg({'year': 'count', 'transactions': 'sum'}).reset_index()[['year', 'transactions']].agg(['mean', 'median'])

,year,transactions
mean,1.39463,2.420099
median,1.00000,1.000000


### form template

In [129]:
q = "select id as form, template from form"
form_templates = redshift_query_read(q, schema='production')

In [131]:
form_templates.tail(3)

,form,template
68805,986317,8
68806,986333,8
68807,986349,6


In [136]:
agg_form_retention = form_retention_donors.groupby(['form'])['year'].agg(['mean', 'median']).reset_index()
agg_form_retention = agg_form_retention.merge(form_templates, on='form')

In [138]:
agg_form_retention.groupby('template')[['mean', 'median']].mean()

,mean,median
template,,
0,1.036051,1.010753
1,1.015515,1.000000
2,1.060437,1.000000
3,1.191389,1.020124
4,1.128123,1.052632
6,1.097964,1.035650
7,1.141097,1.114144
8,1.132405,1.051145
9,1.005465,1.000683


__There doesn't appear to be any significant difference in retention by template__